In [1]:
from torchvision.models import *
import pretrainedmodels

from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
import sys

In [2]:
def arch_summary(arch):
    model = arch(False)
    tot = 0
    for i, l in enumerate(model.children()):
        n_layers = len(flatten_model(l))
        tot += n_layers
        print(f'({i}) {l.__class__.__name__:<12}: {n_layers:<4}layers (total: {tot})')


def get_groups(model, layer_groups):
    group_indices = [len(g) for g in layer_groups]
    curr_i = 0
    group = []
    for layer in model:
        group_indices[curr_i] -= len(flatten_model(layer))
        group.append(layer.__class__.__name__)
        if group_indices[curr_i] == 0:
            curr_i += 1
            print(f'Group {curr_i}:', group)   
            group = []


## 获取数据

In [3]:
root_dir = 'E:\DataSets\DataSets'

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('E:\DataSets\DataSets\garbage_classify_5\info.csv')
df.head()

,img,label,info
0,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_58.jpg,0,00_其他垃圾_一次性快餐盒
1,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_5.jpg,0,00_其他垃圾_一次性快餐盒
2,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_125.jpg,0,00_其他垃圾_一次性快餐盒
3,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_135.jpg,0,00_其他垃圾_一次性快餐盒
4,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_288.jpg,0,00_其他垃圾_一次性快餐盒


In [6]:
pos = 3
means = int(len(df) / 5)
nums = list(range(0, len(df)))

valid_index = nums[pos*means:(pos+1)*means]
train_index = nums[0:pos*means] + nums[(pos+1)*means:]
len(valid_index), len(train_index)

(4514, 18056)

In [7]:
df = df.iloc[:, [0, 1]]
df.head()

,img,label
0,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_58.jpg,0
1,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_5.jpg,0
2,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_125.jpg,0
3,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_135.jpg,0
4,garbage_classify_5/0/00_其他垃圾_一次性快餐盒/img_288.jpg,0


In [8]:
device = torch.device('cuda:1')

In [9]:
tfms = get_transforms(do_flip=False, 
                  flip_vert=True, 
                  max_rotate=10, 
                  max_zoom=1, 
                  max_lighting=None, 
                  max_warp=None, 
                  p_affine=0.75, 
                  p_lighting=None)
tfms[0]

[RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
 RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10, 10)}, p=0.75, resolved={}, do_run=True, is_random=True, use_on_y=True)]

In [10]:
data = (ImageList.from_df(df=df, path=root_dir)
        .split_by_idxs(train_idx=train_index, valid_idx=valid_index)
        .label_from_df()
        .transform(tfms, size=256, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=32, device=device)
        .normalize(imagenet_stats))

## 获取模型

In [11]:
from torch import nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduction='elementwise_mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction is None:
            return F_loss
        else:
            return torch.mean(F_loss)

In [12]:
def se_resnext50(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.se_resnext50_32x4d(pretrained=pretrained)
    return model

In [13]:
learn = cnn_learner(data,se_resnext50, pretrained=True, cut=-2, 
                    split_on=lambda m: (m[0][3], m[1]),
                    metrics=[accuracy, error_rate],
                    callback_fns=[ShowGraph])
learn.loss_fn = FocalLoss()

In [14]:
len(learn.layer_groups)

3

## first stage

In [15]:
path = 'E:\\Projects\\fastai\\5folder\\resnext\\pth{}'.format(pos)

In [16]:
if not os.path.exists(path):
    os.mkdir(path)
path = Path(path)

In [17]:
# learn.freeze()
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [18]:
# lr = 2.09e-3
# learn.fit_one_cycle(5, lr)

In [19]:
# learn.save(path/'stage1')

## two stage

In [20]:
# learn.load(path/'stage1')
# learn.unfreeze()
# learn.lr_find()
# learn.recorder.plot(suggestion=True)

In [21]:
# lr = lr / 5

In [22]:
# learn.fit_one_cycle(20, slice(2e-5, lr))

In [23]:
# learn.save(path/'stage2')

## save ans

In [24]:
learn.load(path/'stage2')

Learner(data=ImageDataBunch;

Train: LabelList (18056 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
0,0,0,0,0
Path: E:\DataSets\DataSets;

Valid: LabelList (4514 items)
x: ImageList
Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256),Image (3, 256, 256)
y: CategoryList
1,1,1,1,1
Path: E:\DataSets\DataSets;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, 

In [25]:
import pandas as pd
import numpy as np

In [26]:
ans = learn.get_preds()
pred = ans[0].numpy()
target = ans[1].numpy().reshape((4514, 1))
ans = np.concatenate([pred, target], axis=1)
df = pd.DataFrame(ans)
df.to_csv(path/'ans.csv', index=False)

In [27]:
preds, targets = learn.TTA()
accuracy(preds, targets)

tensor(0.9233)

In [28]:
preds = preds.data.numpy()
targets = targets.reshape((-1,1))
preds.shape, targets.shape

((4514, 40), torch.Size([4514, 1]))

In [29]:
ans = np.concatenate([preds, targets], axis=1)
ans.shape

(4514, 41)

In [30]:
df  = pd.DataFrame(ans)
df.to_csv(path/'ans.csv', index=False,encoding='utf-8')